In [185]:
using COSMO, LinearAlgebra, Random

In [27]:
# Non-negative. Generally an indefinite matrix, but solve for
# PSD X as max(Tr AX), this PSD program is a linear program with
# conic constraint.
A = [1 0.1 0.2 0.05; 0.2 1 0.5 0.01; 0.3 0.2 1 0.05; 0.01 0.5 0.05 1]
k = 2
n = size(A, 2)
N = n * n
A

4×4 Matrix{Float64}:
 1.0   0.1  0.2   0.05
 0.2   1.0  0.5   0.01
 0.3   0.2  1.0   0.05
 0.01  0.5  0.05  1.0

In [55]:
Matrix(1.0I, N, N)[begin:n:end, :]

4×16 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0  0.0  0.0

In [64]:
[zeros(1, N) 1; Matrix(1.0I, N, N)[begin:n:end, :] zeros(n, 1)]

5×17 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0

In [182]:
model = COSMO.Model()
psd_c = COSMO.Constraint(Matrix(1.0I, N, N), zeros(N), COSMO.PsdCone)
box_c = COSMO.Constraint(
    Matrix(1.0I, N, N), zeros(N), COSMO.Box(zeros(N), ones(N)))
# Elements equal to 1 or 0. Sum of squares (||v||_2^2) equal to k.
l2_c = COSMO.Constraint(
    vec(Matrix(1.0I, n, n))', [-Float64(k)], COSMO.ZeroSet(1))
# One row: The diag entry may be zero or nonzero.
row_c = repeat(Matrix(-1.0I, n, n), inner=(1, n))
for i = 1:n
    row_c[i, 1 + (n+1)*(i-1)] = k-1
end
row_c = COSMO.Constraint(row_c, zeros(n), COSMO.ZeroSet(n))
assemble!(model, zeros(N, N), -vec(A), [psd_c, box_c, l2_c, row_c])

In [183]:
result = COSMO.optimize!(model)

>>> COSMO - Results
Status: 

Solved
Iterations: 25 (incl. 0 safeguarding iterations)
Optimal Objective: -2.7
Runtime: 0.81ms
Setup Time: 0.14ms

Avg Iter Time: 0.02ms

In [184]:
reshape(result.x, n, n)

4×4 Matrix{Float64}:
 -7.71537e-9  -2.84948e-9  -2.05334e-9   2.04365e-9
 -3.01474e-9   1.0          1.0         -3.69062e-9
 -7.21494e-9   1.0          1.0          2.50193e-9
  2.51798e-9   1.4283e-8    1.81136e-9   8.50403e-10

In [197]:
# Non-negative. Generally an indefinite matrix, but solve for
# PSD X as max(Tr AX), this PSD program is a linear program with
# conic constraint.
n = 101
A = abs.(randn(n, n))
k = 3
N = n * n;

In [201]:
model = COSMO.Model()
psd_c = COSMO.Constraint(Matrix(1.0I, N, N), zeros(N), COSMO.PsdCone)
box_c = COSMO.Constraint(
    Matrix(1.0I, N, N), zeros(N), COSMO.Box(zeros(N), ones(N)))
# Elements equal to 1 or 0. Sum of squares (||v||_2^2) equal to k.
l2_c = COSMO.Constraint(
    vec(Matrix(1.0I, n, n))', [-Float64(k)], COSMO.ZeroSet(1))
# One row: The diag entry may be zero or nonzero.
row_c = repeat(Matrix(-1.0I, n, n), inner=(1, n))
for i = 1:n
    row_c[i, 1 + (n+1)*(i-1)] = k-1
end
row_c = COSMO.Constraint(row_c, zeros(n), COSMO.ZeroSet(n))
assemble!(model, zeros(N, N), -vec(A), [psd_c, box_c, l2_c, row_c])

In [202]:
result = COSMO.optimize!(model)

>>> COSMO - Results
Status: 

Solved
Iterations: 1500 (incl. 0 safeguarding iterations)
Optimal Objective: -16.38
Runtime: 7741.77ms
Setup Time: 21.55ms

Avg Iter Time: 5.14ms

In [209]:
maximum(model.δx)

2.3209046042693693e-5